# 통계적 검정 실습

## 패키지 import

In [1]:
# 패키지 import
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import math
from scipy import stats

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# os에 따른 Font 깨짐 제거를 위해 Font 지정
import os
if os.name == 'nt':
    font_family = 'Malgun Gothic'
    
# - 값이 깨지는 문제 해결을 위해 파라미터값 설정
sns.set(font=font_family, rc={'axes.unicode_minus': False})

## 1. t검정

### 1) 일표본 t-검정
- 학생 14명 대상
- 귀무가설: 아이리포 중학교 1학년의 평균 키는 175cm
- 대립가설: 아이리포 중학교 1학년의 평균 키는 175cm 아님

In [2]:
# 모집단 데이터
data = [177.3, 182.7, 169.6, 160, 180.3, 179.4, 178.5, 177.2, 181.8, 176.5, 190, 185, 167, 171]

In [3]:
# 데이터의 정규성 검정: p_value > 0.05이면 정규성을 따름
stats.shapiro(data)

ShapiroResult(statistic=0.9641273021697998, pvalue=0.7900116443634033)

stats.ttest_1samp로 검정통계량 계산

In [4]:
# 유의수준 0.05
m_a = 0.05

# 알려진 평균 확인
s_mu = 175
one_sample_result = stats.ttest_1samp(data, popmean = s_mu)

print('t검정 통계량 = %.3f, p_value = %.3f' %(one_sample_result))

t검정 통계량 = 0.903, p_value = 0.383


검정통계량 직접 계산

In [5]:
# 표본 수
n = len(data)
print(len(data))
hysis_mu = np.mean(data)
print(np.mean(data))

# 알려진 평균
s_mu = 175

# 표본표준편차
s_sigma = np.std(data, ddof=1)
print(s_sigma)

# 통계량 t값 계산
t_value = (hysis_mu - s_mu) / (s_sigma / np.sqrt(n))
print('t검정 통계량 :', round(t_value, 3))

# p_value 계산
print('p_value :', round((1 - stats.t(df=13).cdf(t_value)) * 2, 3))

14
176.87857142857143
7.782443611837389
t검정 통계량 : 0.903
p_value : 0.383


In [6]:
# 유의수준 0.05
print('유의수준 :', m_a)

# 기각값
t_95 = stats.t.ppf(1 - (1 - 0.95) / 2, df = n-1)

print('기각값 :', t_95)

유의수준 : 0.05
기각값 : 2.1603686564610127


결론
- 검정통계량 검정: 0.903은 채택역에 속함
- p-value 검정: 0.05(유의수준) < 0.383(p-value), 귀무가설 채택
- 아이리포 중학교 1학년의 평균 키는 175cm

### 2) 대응표본 t-검정
- 환자 10명 대상
- 귀무가설: 수면영양제를 복용하기 전과 후의 평균 수면시간에는 차이가 없음(D=0)
- 대립가설: 수면영양제를 복용하기 전과 후의 평균 수면시간 차이는 0보다 작음(D<0)

In [7]:
# 검정 데이터 관측치
before_data = [7, 3, 4, 5, 2, 1, 6, 6, 5, 4]
after_data = [8, 4, 5, 6, 2, 3, 6, 8, 6, 5]

check = ['before', 'after']

# 데이터프레임 merge
check = [check[j] for j in range(2) for i in range(10)]
data = pd.DataFrame({'when': check, 'score': before_data + after_data})

In [8]:
data.head()

,when,score
0,before,7
1,before,3
2,before,4
3,before,5
4,before,2


In [9]:
# 정규성 검정: 자료의 모집단 분포는 정규분포를 따름
from scipy.stats import shapiro

normal_before = shapiro(before_data)
normal_after = shapiro(after_data)

print(normal_before)
print(normal_after)

# p-value 모두 0.05보다 크기 때문에 정규성 만족

ShapiroResult(statistic=0.9644594192504883, pvalue=0.8352694511413574)
ShapiroResult(statistic=0.9456835985183716, pvalue=0.6177982091903687)


In [10]:
# 등분산성 검정: 모든 집단의 모분산은 동일함
from scipy.stats import levene
print(levene(before_data, after_data))

from scipy.stats import bartlett
print(bartlett(before_data, after_data))

# p-value가 0.05보다 커서 등분산성 만족

LeveneResult(statistic=0.0, pvalue=1.0)
BartlettResult(statistic=0.007785808167159078, pvalue=0.9296881301038368)


In [11]:
# 대응표본 t검정 수행
paired_result = stats.ttest_rel(before_data, after_data)

In [12]:
print('검정통계량 = %.3f, pvalue = %.3f' %paired_result)

검정통계량 = -4.743, pvalue = 0.001


결론
- 대응표본 t검정 수행 결과, t검정 통계량 값은 -4.743, p-value는 0.001
- p-value 검정: 0.05(유의수준) > 0.001(p-value), 귀무가설 기각
- 수면영양제를 복용하기 전과 후의 평균 수면시간 차이는 통계적으로 유의하며, 영양제를 복용한 후 수면시간이 줄었다는 결론 내릴 수 있음

### 3) 독립표본 t-검정
- 12일동안 서울, 대구 두 지역의 낮 최고기온 측정
- 귀무가설: a, b 두 지역에 따른 겨울 낮 최고기온은 차이가 없음
- 대립가설: a, b 두 지역에 따른 겨울 낮 최고기온은 차이가 있음

In [13]:
# 데이터프레임
seoul = [-1, 0, 3, 4, 1, 3, 3, 1, 1, 3, 2, 4]
daegu = [6, 6, 8, 8, 11, 11, 10, 8, 8, 9, 7, 10]
region = ['seoul'] * 12 + ['daegu'] * 12
data = pd.DataFrame({'region': region, 'temp': seoul + daegu})

In [14]:
data.head()

,region,temp
0,seoul,-1
1,seoul,0
2,seoul,3
3,seoul,4
4,seoul,1


In [15]:
# 정규성 검정: 자료의 모집단 분포는 정규분포를 따름

normal_seoul = shapiro(seoul)
normal_daegu = shapiro(daegu)
print(normal_seoul)
print(normal_daegu)

# 결과: p-value > 0.05, 정규성을 만족

ShapiroResult(statistic=0.9178412556648254, pvalue=0.2685365378856659)
ShapiroResult(statistic=0.9212772846221924, pvalue=0.2966691553592682)


In [16]:
# 등분산성 검정: 모든 집단의 모분산은 동일함

# levene 등분산성 검정
print(levene(seoul, daegu))

# bartlett 등분산성 검정
print(bartlett(seoul, daegu))

# 결과: p-value > 0.05, 등분산성 만족

LeveneResult(statistic=0.0, pvalue=1.0)
BartlettResult(statistic=0.07093015354702552, pvalue=0.7899872613272646)


In [17]:
# 독립표본 t검정
ttests_result = stats.ttest_ind(seoul, daegu, equal_var = True)
print('t검정 통계량: %.3f, p-value = %.15f' %(ttests_result))

t검정 통계량: -9.562, p-value = 0.000000002711180


결론
- 독립표본 t검정 수행결과, t검정 통계량 값은 -9.562, p-value는 0.000000002711180
- p-value 검정: 0.05(유의수준) > 0.000000002711180(p-value), 귀무가설 기각
- 서울과 대구, 두 지역의 겨울 낮 최고기온에는 통계적으로 유의한 차이가 존재한다는 결론 내릴 수 있음